# DER calculation
This file is used to calculate Diarization Error Rate. It is calculated as follows:
1. For each speaker, we calculate the number of false alarms, missed detections and overlapped speech.
2. We sum up the number of false alarms, missed detections and overlapped speech.
3. We divide the sum by the total number of speaker speech segments.
4. The result is the DER.

The formula is as follows:
$$
DER = \frac{FA + MISS + OVER}{N_{spk}}
$$

Where:
- $FA$ is the number of false alarms
- $MISS$ is the number of missed detections
- $OVER$ is the number of overlapped speech
- $N_{spk}$ is the total number of speaker speech segments

The code below is used to calculate the DER.

In [21]:
import pandas as pd
from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate
import os
from tqdm import tqdm

# Paths to the different RTTM files
RTTM_TRUTH = "../Dataset/RTTMs/Test"
RTTM_ORACLE_VAD = "../Results/Oracle_vad/Test"


# Load the RTTM files
def load_rttm(filename):
    """Load RTTM file and convert it to pyannote.core.Annotation"""
    annotation = Annotation()
    with open(filename, "r") as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) < 9:
                continue
            file_id, channel, start_time, duration, _, _, *speaker_parts = parts
            speaker = " ".join(speaker_parts)
            start_time = float(start_time)
            duration = float(duration)
            end_time = start_time + duration
            segment = Segment(start_time, end_time)
            annotation[segment] = speaker
    return annotation


# Initialize the diarization error rate metric
der_metric = DiarizationErrorRate()

# Store results
results = []

# Iterate over files in RTTM_ORACLE_VAD directory
for file_name in tqdm(os.listdir(RTTM_ORACLE_VAD)):
    path_truth = os.path.join(RTTM_TRUTH, file_name)
    path_oracle_vad = os.path.join(RTTM_ORACLE_VAD, file_name)

    if os.path.exists(path_truth) and os.path.exists(path_oracle_vad):
        ground_truth = load_rttm(path_truth)
        prediction = load_rttm(path_oracle_vad)

        # Calculate DER and components manually
        detailed = der_metric(ground_truth, prediction, detailed=True)
        print(detailed)
        der = detailed["diarization error rate"]
        missed_speech = detailed["missed speech"]
        speaker_error = detailed["speaker error"]
        false_alarm = detailed["false alarm"]
        total_duration = detailed["total"]

        # Append results
        results.append(
            {
                "File": file_name,
                "DER": der,
                "Miss Duration": missed_speech / total_duration,
                "Speaker Error Duration": speaker_error / total_duration,
                "False Alarm Duration": false_alarm / total_duration,
            }
        )
    else:
        print(f"File {file_name} not found")


# Convert results to DataFrame for better readability
results_df = pd.DataFrame(results)
print(results_df)

# If you want to save the results to a CSV file
results_df.to_csv("der_results.csv", index=False)

  0%|          | 0/232 [00:00<?, ?it/s]c:\Users\rakin\anaconda3\Lib\site-packages\pyannote\metrics\utils.py:200: UserWarning: 'uem' was approximated by the union of 'reference' and 'hypothesis' extents.
  warnings.warn(
  0%|          | 0/232 [00:00<?, ?it/s]

{'missed detection': 0.0, 'confusion': 32.65, 'false alarm': 59.300000000000004, 'correct': 2430.3499999999995, 'total': 2463.0, 'diarization error rate': 0.03733252131546894}


KeyError: 'missed speech'